In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional, Callable, Any, Tuple
from timm.models.layers import to_2tuple,to_3tuple

In [32]:
class PatchEmbed(nn.Module):
    """ patch splitting 

    Args:
        img_size (int): 输入图片的尺寸
        patch_size (int): Patch token的尺度. Default: 4.
        in_chans (int): 输入通道的数量
        embed_dim (int): 线性投影后输出的维度
        norm_layer (nn.Module, optional): 这可以进行设置，在本层中设置为了None
    """

    def __init__(self, img_size=224, patch_size=4, in_chans=512, embed_dim=96, norm_layer=None):
        super().__init__()
        # 先将img_size、patch_size转化为元组模式(224 , 224) 、 (4 , 4)
        img_size = to_2tuple(img_size) 
        patch_size = to_2tuple(patch_size)
        # 计算出 Patch token在长宽方向上的数量
        patches_resolution = [img_size[0] // patch_size[0], img_size[1] // patch_size[1]]
        self.img_size = img_size
        self.patch_size = patch_size
        self.patches_resolution = patches_resolution
        # 计算出patch的数量利用Patch token在长宽方向上的数量相乘的结果
        self.num_patches = patches_resolution[0] * patches_resolution[1]
        self.in_chans = in_chans
        self.embed_dim = embed_dim

        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)
        # 判断是否使用norm_layer，在这里我们没有应用
        if norm_layer is not None:
            self.norm = norm_layer(embed_dim)
        else:
            self.norm = None

    def forward(self, x):
    	# 解析输入的维度
        B, C, H, W = x.shape
        # 判断图像是否与设定图像一致，如果不一致会报错
        assert H == self.img_size[0] and W == self.img_size[1], \
            f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]})."
        # 经过一个卷积层来进行一个线性变换，并且在第二个维度上进行一个压平操作，维度为(B, C, Ph*Pw),后在进行一个维与二维的一个转置，维度为：(B Ph*Pw C)
        x = self.proj(x).flatten(2).transpose(1, 2)  
        if self.norm is not None:
            x = self.norm(x)
        return x


In [25]:
class PatchMerging(nn.Module):
    r""" Patch合并.

    Args:
        input_resolution (tuple[int]): 输入特征的分辨率.
        dim (int): 输入通道的数量
        norm_layer (nn.Module, optional): 定义Normalization layer.  Default: nn.LayerNorm
    """

    def __init__(self, input_resolution, dim, norm_layer=nn.LayerNorm):
        super().__init__()
        self.input_resolution = input_resolution
        self.dim = dim
        # 通过一个线性层将4C降为2C
        self.reduction = nn.Linear(4 * dim, 2 * dim, bias=False)
        self.norm = norm_layer(4 * dim)

    def forward(self, x):
        """
        x: B, H*W, C
        """
        # 解析输入图像的分辨率，即输入图像的长宽
        H, W = self.input_resolution
        # 解析输入图像的维度
        B, L, C = x.shape
        # 判断L是否与H * W一致，如不一致会报错
        assert L == H * W, "input feature has wrong size"
        # 判断输入图像的长宽是否可以被二整除，因为我们是通过2倍来进行下采样的
        assert H % 2 == 0 and W % 2 == 0, f"x size ({H}*{W}) are not even."
        # 将xreshape为维度：(B, H, W, C)
        x = x.view(B, H, W, C)
        # 切片操作，通过切片操作将将相邻的2*2的patch进行拼接
        x0 = x[:, 0::2, 0::2, :]  # B H/2 W/2 C
        
        x1 = x[:, 1::2, 0::2, :]  # B H/2 W/2 C
        x2 = x[:, 0::2, 1::2, :]  # B H/2 W/2 C
        x3 = x[:, 1::2, 1::2, :]  # B H/2 W/2 C
        # 将合并好的patch通过c维进行拼接
        x = torch.cat([x0, x1, x2, x3], -1)  # B H/2 W/2 4*C
        # 将x的维度重置为B H/2*W/2 4*C
        x = x.view(B, -1, 4 * C)  # B H/2*W/2 4*C
        x = self.norm(x)
        # 通过一个线性层进通道降维
        x = self.reduction(x)

        return x


In [26]:
class SwinTransformerBlock(nn.Module):
    """ Swin Transformer Block.

    Args:
        dim (int): 输入维度的数量
        input_resolution (tuple[int]): 输入图像的分辨率
        num_heads (int): 应用注意力头的数量
        window_size (int): 窗口的尺寸
        shift_size (int): SW-MSA的循环位移的大小，默认为零
        mlp_ratio (float): mlp的隐层的比例,默认为零
        qkv_bias (bool, optional): 是否应用位移偏量，具体实现在下文
        qk_scale (float | None, optional): 如果设置，默认qk覆盖head_dim ** -0.5。
        drop (float, optional): Dropout比例，防止过拟合的操作，默认为零
        attn_drop (float, optional): AttentionDropout比例，默认为零
        drop_path (float, optional): 随机深度比率，默认为0.0
        act_layer (nn.Module, optional): 激活函数层，默认为GELU
        norm_layer (nn.Module, optional): Normalization层， 默认为 nn.LayerNorm
    """

    def __init__(self, dim, input_resolution, num_heads, window_size=7, shift_size=0,
                 mlp_ratio=4., qkv_bias=True, qk_scale=None, drop=0., attn_drop=0., drop_path=0.,
                 act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.dim = dim
        self.input_resolution = input_resolution
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        self.mlp_ratio = mlp_ratio
        if min(self.input_resolution) <= self.window_size:
            # 如果Windows大小大于输入分辨率，则不分区Windows
            self.shift_size = 0
            self.window_size = min(self.input_resolution)
        # 如果Windows大小不符合指定的数值，则报错
        assert 0 <= self.shift_size < self.window_size

        self.norm1 = norm_layer(dim)
        # 窗口注意力，在接下来进行详细说明
        self.attn = WindowAttention_acmix(
                dim, window_size=to_2tuple(self.window_size), num_heads=num_heads,
                qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)

        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

        if self.shift_size > 0:
            # 判断是否使用窗口移位，如果使用窗口移位，会使用mask windows，会在下文中进行分开讲解
            H, W = self.input_resolution
            img_mask = torch.zeros((1, H, W, 1))
            h_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            w_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            cnt = 0
            for h in h_slices:
                for w in w_slices:
                    img_mask[:, h, w, :] = cnt
                    cnt += 1

            mask_windows = window_partition(img_mask, self.window_size)  
            mask_windows = mask_windows.view(-1, self.window_size * self.window_size)
            attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
            attn_mask = attn_mask.masked_fill(attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))
        else:
            attn_mask = None
		# 向模块添加一个attn_mask的持久缓冲区
        self.register_buffer("attn_mask", attn_mask)

    def forward(self, x):
    	#解析输入图像的分辨率
        H, W = self.input_resolution
        # 解析输入的维度
        B, L, C = x.shape
        # 判断L和H * W是否一致
        assert L == H * W, "input feature has wrong size"
		# 进行原始的身份映射
        shortcut = x
        # 首先经过以一个LN层
        x = self.norm1(x)
        # 将维度重置为(B, H, W, C)
        x = x.view(B, H, W, C)

        # 判断是否进行循环移位
        if self.shift_size > 0:
            shifted_x = torch.roll(x, shifts=(-self.shift_size, -self.shift_size), dims=(1, 2))
        else:
            shifted_x = x

        shifted_x = self.attn(shifted_x, H, W, mask=self.attn_mask)

        # 循环移位进行恢复
        if self.shift_size > 0:
            x = torch.roll(shifted_x, shifts=(self.shift_size, self.shift_size), dims=(1, 2))
        else:
            x = shifted_x
        x = x.view(B, H * W, C)

        # FFN
        x = shortcut + self.drop_path(x)
        x = x + self.drop_path(self.mlp(self.norm2(x)))

        return x


In [27]:
 class WindowAttention(nn.Module):
    """ 基于相对位置偏差的窗口多头自注意(W-MSA)模块。
        它同时支持W-MSA和SW-MSA

    Args:
        dim (int): 输入通道的数量
        window_size (tuple[int]): window的长和宽.
        num_heads (int): attention heads的数量.
        qkv_bias (bool, optional): 如果为True，则向query, key, value添加一个可学习的偏差。默认值: True
        qk_scale (float | None, optional): 如果设置，覆盖head_dim ** -0.5的默认qk值
        attn_drop (float, optional): attention weight丢弃率，默认: 0.0
        proj_drop (float, optional): output的丢弃率. 默认:: 0.0
    """
    # 实现W-MSA SW-MSA
    def __init__(self, dim, window_size, num_heads, qkv_bias=True, qk_scale=None, attn_drop=0., proj_drop=0.):
        super(WindowAttention, self).__init__()

        self.dim = dim
        self.window_size = window_size ## Mh, Mw
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5
        # 定义相对位置偏差的参数表 (2*Mh-1 * 2*Mw-1, nH)
        self.relative_positive_bias_table = nn.Parameter(
            torch.zeros((2 * window_size[0] - 1) * (2 * window_size[1] - 1), num_heads) 
        )
        # 获取窗口内每个token的成对相对位置索引
        # 第一行为feature map中每一个像素对应的行标（x）
        # 第二行为feature map中每一个像素对应的列标（y）
        coords_h = torch.arange(self.window_size[0])
        coords_w = torch.arange(self.window_size[1])
        # 拼接成横纵坐标
        coords = torch.stack(torch.meshgrid([coords_h, coords_w])) # [2, Mh, Mw]
        # 沿着Mh这一维度进行展平
        coords_flatten = torch.flatten(coords, 1) # [2, Mh*Mw] 绝对位置索引
        # [2, Mh*Mw, 1] - [2, 1, Mh*Mw]
        # [2, Mh*Mw, Mh*Mw] 得到相对位置索引的矩阵。 以每一个像素作为参考点 - 当前feature map/window当中所有的像素点绝对位置索引 = 得到相对位置索引的矩阵
        # coords_flatten[:, :, None] 按w维度 每一行的元素复制
        # coords_flatten[:, None, :] 按h维度 每一行元素整体复制
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]
        # permute： 将窗口中按每个像素求得的相对位置索引 组成矩阵
        relative_coords = relative_coords.permute(1, 2, 0).contiguous() # [Mh, Mw, 2]
        # 二元索引->一元索引
        relative_coords[:, :, 0] += self.window_size[0] - 1
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 0] *= 2 * self.window_size[1] - 1
        relative_position_index = relative_coords.sum(-1) # [Mh*Mw, Mh*Mw]
        # 放到模型缓存中
        self.register_buffer('relative_position_index', relative_position_index)

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        nn.init.trunc_normal_(self.relative_positive_bias_table, std=.02)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, mask: Optional[torch.Tensor] = None):
        # 解析输入维度[batch_size * num_windows, Mh*Mw, total_embed_dim]
        B_, N, C = x.shape
        # qkv: -> [batch_size * num_windows, Mh*Mw, 3 * total_embed_dim]
        # reshape: -> [batch_size * num_windows, Mh*Mw, 3, num_heads, embed_dim_per_head]
        # permute: -> [3, batch_size * num_windows, num_heads, Mh*Mw, embed_dim_per_head]
        qkv = self.qkv(x).reshape(B_, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        # [batch_size * num_windows, num_heads, Mh*Mw, embed_dim_per_head]
        q, k, v = qkv.unbind(0)

        q = q * self.scale
        # transpose: -> [batch_size * num_windows, num_heads,embed_dim_per_head, Mh*Mw]
        # @: multiply: -> [batch_size * num_windows, num_heads, Mh*Mw, Mh*Mw]
        attn = (q @ k.transpose(-2, -1))
        # self.relative_positive_bias_table.view: -> [Mh*Mw*Mh*Mw, num_head] -> [Mh*Mw, Mh*Mw, num_head]
        relative_position_bias = self.relative_positive_bias_table[self.relative_position_index.view(-1)].view(
            self.window_size[0] * self.window_size[1], self.window_size[0] * self.window_size[1], -1)
        relative_position_bias = relative_position_bias.permute(2, 0, 1).contiguous() # [num_head, Mh*Mw, Mh*Mw]
        # [batch_size * num_windows, num_heads, Mh * Mw, Mh * Mw]
        attn = attn + relative_position_bias.unsqueeze(0)

        if mask is not None:
            # mask: [num_windows, Mh*Mw, Mh*Mw]
            num_window = mask.shape[0]
            # view: [batch_size, num_windows, num_heads, Mh * Mw, Mh * Mw]
            # mask: [1, num_windows, 1, Mh*Mw, Mh*Mw]
            attn = attn.view(B_ // num_window, num_window, self.num_heads, N, N) + mask.unsqueeze(1).unsqueeze(0)
            # [batch_size*num_windows, num_heads, Mh * Mw, Mh * Mw]
            attn = attn.view(-1, self.num_heads, N, N)
            attn = self.softmax(attn)
        else:
            attn = self.softmax(attn)

        attn = self.attn_drop(attn)
        # @: [batch_size*num_windows, num_heads, Mh * Mw, embed_dim_per_head]
        # transpose: [batch_size*num_windows, Mh * Mw, num_heads, embed_dim_per_head]
        # reshape: [num_windows, Mh * Mw, num_heads*embed_dim_per_head]
        x = (attn @ v).transpose(1, 2).reshape(B_, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)

        return x



In [28]:
# 指定window大小，重新划分window
def window_partition(x, window_size: int):
    # 将feature map(image mask) 按照 window_size的大小 划分成一个个没有重叠的window
    B, H, W, C = x.shape
    # [B, H//M, MH, W//M, MW, C] MH: 为窗口H MW:为窗口W
    x = x.view(B, H // window_size, window_size, W // window_size, window_size, C)
    # permute: [B, H//M, MH, W//M, MW, C] -> [B, H//M, W//M, MH, MW, C]
    # contiguous(): 变为内存连续的数据
    # view： [B, H//M, W//M, M, M, C] -> [B * window_num, MH, MW, C] 
    windows = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(-1, window_size, window_size, C)
    # 最后会返回一个带有窗口数量以及窗口长宽的一个张量
    return windows

    if self.shift_size > 0:
            # 判断是否使用SW-MSA，如果使用窗口移位，会使用mask windows
            # 解析输入图像的分辨率
            H, W = self.input_resolution
            img_mask = torch.zeros((1, H, W, 1))  # 1 H W 1
            # 其中的切片操作划分后的结构如上图，切出每个窗口中分别具有相似元素的位置
            h_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            w_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            # 开始切片，并且将相同的位置附上相同的数值            
            cnt = 0
            for h in h_slices:
                for w in w_slices:
                    img_mask[:, h, w, :] = cnt
                    cnt += 1
            # nW, window_size, window_size, 1
            mask_windows = window_partition(img_mask, self.window_size) 
            # [B * window_num * C, MH*MW]
            mask_windows = mask_windows.view(-1, self.window_size * self.window_size)
            #  mask_windows.unsqueeze(1) 将每个窗口的行向量复制MH*MW次 
            #  mask_windows.unsqueeze(2) 将每个窗口的行向量中每个元素 复制MH*MW次             
            attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
            # 同一区域为0 不同区域为非0数。 得到当前窗口中对应某一个像素 所采用的attention mask。
            attn_mask = attn_mask.masked_fill(attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))
        


In [29]:
class SwimTransformerBlock(nn.Module):
    def __init__(self, dim, num_heads, window_size=7, shift_size=0., mlp_ratio=4., qkv_bias=True, drop=0., attn_drop=0., drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super(SwimTransformerBlock, self).__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        self.mlp_ratio = mlp_ratio

        self.norm1 = norm_layer(dim)
        self.attn = WindowAttention(
            dim=dim, window_size=(self.window_size, self.window_size), num_heads=num_heads,
            qkv_bias=qkv_bias, attn_drop=attn_drop, proj_drop=drop
        )
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = (int(dim * mlp_ratio))
        self.mlp = MLP(in_features=dim, hidden_features=mlp_hidden_dim, act=act_layer, drop=drop)

    def forward(self, x, attn_mask):
        H, W = self.H, self.W # feature map H W
        B, L, C = x.shape # L = H * W

        shortcut = x
        x = self.norm1(x)
        x = x.view(B, H, W, C)

        x_r = (self.window_size - W % self.window_size) % self.window_size
        x_d = (self.window_size - H % self.window_size) % self.window_size
        x = F.pad(x, (0, 0, 0, x_r, 0, x_d))
        _, Hp, Wp, _ = x.shape # Hp Wp代表padding后的H W

        if self.shift_size > 0.:
            # SW-MSA 从上往下 从左往右
            shifted_x = torch.roll(x, shifts=(-self.shift_size, -self.shift_size), dims=(1, 2)) # 上面的shift size移动到下面 左边移动右边
        else:
            # W-MSA
            shifted_x = x
            attn_mask = None

        # 特征图切成小窗口
        x_windows = window_partition(shifted_x, self.window_size) # [B * window_num, MH, MW, C]
        x_windows = x_windows.view(-1, self.window_size * self.window_size, C) # [B * window_num, MH*MW, C]

        # W-MSA SW-MSA
        attn_windows = self.attn(x_windows, mask=attn_mask)

        # 小窗口合并成特征图
        attn_windows = attn_windows.view(-1, self.window_size, self.window_size, C) # [B * window_num, MH, MW, C]
        shifted_x = window_reverse(attn_windows, self.window_size, Hp, Wp) # [B, H, W, C]
        # SW-MSA后还原数据 从下往上 从右往左
        if self.shift_size > 0:
            x = torch.roll(shifted_x, shifts=(self.shift_size, self.shift_size), dims=(1, 2))
        else:
            x = shifted_x
        # 移除padding
        if x_r > 0 or x_d > 0:
            x = x[:, :H, :W, :].contiguous()

        x = x.view(B, H * W, C)

        x = shortcut + self.drop_path(x)
        x = x + self.drop_path(self.mlp(self.norm2(x)))

        return x



In [3]:
import torch
import torchvision
from torchvision.models import swin_transformer

ImportError: cannot import name 'swin_transformer' from 'torchvision.models' (D:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\__init__.py)